## Using a tool

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
tool.name

'wikipedia'

In [4]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [5]:
tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [6]:
tool.return_direct

False

In [7]:
tool.run({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

## Custom tools

In [1]:
from langchain.tools import tool


@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search("What's the best application framework for LLMs?")

'LangChain'

In [2]:
from langchain.pydantic_v1 import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search("How do we implement LLM apps?")

'LangChain'

In [5]:
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return "LangChain"

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

search = CustomSearchTool()
search("What's the most popular tool for writing LLM apps?")

'LangChain'

In [6]:
from langchain.tools import StructuredTool

def search_function(query: str):
    return "LangChain"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
)
search("Which framework has hundreds of integrations to use with LLMs?")

'LangChain'

In [9]:
 class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
) 

calculator.run(dict(a=1_000_000_000, b=2))

2000000000

## Error handling

In [12]:
from langchain_core.tools import ToolException

def search_tool(s: str):
    raise ToolException("The search tool is not available.")

search = StructuredTool.from_function(
    func=search_tool,
    name="Search_tool",
    description="A bad tool",
    handle_tool_error=True,
)
search("Search the internet and compress everything into a paragraph!")

'The search tool is not available.'